In [7]:
# Step 1: Import necessary things
from langchain_core.messages import HumanMessage
from chatbot.langgraph_flow import gig_chatbot_graph  # Adjust path if needed


In [8]:
# Cell 1: Imports
from langgraph.graph import StateGraph
from langgraph.graph.graph import END
from langchain_core.messages import HumanMessage, AIMessage
from typing import Annotated, List, TypedDict

In [9]:
# Cell 2: Define the state
class ChatbotState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], lambda x: x[-10:]]


In [10]:
# Cell 3: Define the start node
def start_node(state: ChatbotState) -> ChatbotState:
    print("📍 Running start_node")
    messages = state["messages"]
    messages.append(AIMessage(content="Hi there! I’m your assistant. How can I help you today?"))
    return {"messages": messages}

In [11]:
# Cell 4: Build the graph
def build_graph():
    builder = StateGraph(ChatbotState)

    # Define the nodes
    builder.add_node("start", start_node)

    # Entry point
    builder.set_entry_point("start")

    # End the graph after this node for now
    builder.add_edge("start", END)

    return builder.compile()

In [14]:
# Cell 5: Run the graph
if __name__ == "__main__":
    graph = build_graph()

    # Example user input
    user_input = HumanMessage(content="Can you help me register an event?")

    # Initial state
    state = {"messages": [user_input]}

    # Run the graph
    final_state = graph.invoke(state)

    print("✅ Final State:")
    for msg in final_state["messages"]:
        print(f"{msg.type.upper()}: {msg.content}")

KeyError: '__start__'

In [27]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Optional

class ChatState(TypedDict):
    user_input: str
    intent: Optional[str]

def router_node(state: ChatState) -> ChatState:
    print("🔀 Router received:", state)
    return state

def route_from_intent(state: ChatState) -> str:
    intent = (state.get("intent") or "").strip()
    allowed = {"register_event", "rsvp_help", "analytics_guide", "gig_intro"}
    return intent if intent in allowed else "fallback"

def register_event(state: ChatState) -> ChatState:
    print("📌 register_event:", state); return state

def rsvp_help(state: ChatState) -> ChatState:
    print("📌 rsvp_help:", state); return state

def analytics_guide(state: ChatState) -> ChatState:
    print("📌 analytics_guide:", state); return state

def gig_intro(state: ChatState) -> ChatState:
    print("📌 gig_intro:", state); return state

def fallback(state: ChatState) -> ChatState:
    print("⚠️ fallback:", state); return state

def build_graph():
    builder = StateGraph(ChatState)

    # nodes
    builder.add_node("router", router_node)
    builder.add_node("register_event", register_event)
    builder.add_node("rsvp_help", rsvp_help)
    builder.add_node("analytics_guide", analytics_guide)
    builder.add_node("gig_intro", gig_intro)
    builder.add_node("fallback", fallback)

    # >>> critical entry wiring <<<
    builder.add_edge(START, "router")

    # conditional routing
    builder.add_conditional_edges(
        "router",
        route_from_intent,
        {
            "register_event": "register_event",
            "rsvp_help": "rsvp_help",
            "analytics_guide": "analytics_guide",
            "gig_intro": "gig_intro",
            "fallback": "fallback",
        },
    )

    # leaves go to END
    for node in ["register_event", "rsvp_help", "analytics_guide", "gig_intro", "fallback"]:
        builder.add_edge(node, END)

    return builder.compile()

gig_chatbot_graph = build_graph()

# --- Test
print("---- Test 1: known intent ----")
state = {"user_input": "I want to sign up", "intent": "register_event"}
for event in gig_chatbot_graph.stream(state):
    print("➡️", event)

print("\n---- Test 2: unknown intent ----")
state = {"user_input": "what is this?", "intent": "unknown"}
for event in gig_chatbot_graph.stream(state):
    print("➡️", event)


---- Test 1: known intent ----


KeyError: '__start__'

In [26]:
print("---- Test 1: known intent ----")
state = {"user_input": "I want to sign up", "intent": "register_event"}
for event in gig_chatbot_graph.stream(state):
    print("➡️ Event:", event)

print("\n---- Test 2: unknown intent ----")
state = {"user_input": "what is this?", "intent": "unknown"}
for event in gig_chatbot_graph.stream(state):
    print("➡️ Event:", event)


---- Test 1: known intent ----


KeyError: '__start__'